# Example Usage of EQTransformer from seisbench
This notebook is mostly an uncommented version of the “03a_training_phasenet” notebook from the seisbench repository.

In [46]:
import seisbench.generate as sbg
import seisbench.models as sbm

from seisbench.data import WaveformDataset
from seisbench.models import EQTransformer
from seisbench.util import worker_seeding

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

import pandas as pd
from utils import preprocess


Load the 100Samples dataset from the EQTransformer repository but transformed into seisbench-compatible format.

Rename the label columns.

Perform (minimal) data augmentation.  I assume (but did not test) that `ChangeDtype` is necessary for the code to run; `ProbabilisticLabeller` provides the probability curves below, but I do not know how they are calculated.

In [124]:
batch_size = 1000 #
num_workers = 24  # The number of threads used for loading data

data = WaveformDataset("data/STEAD/example/seisbench")

train_loader, dev_loader, test_loader = preprocess(data, batch_size, num_workers)


In [ ]:
from swag.posteriors import SWAG

dataset = test_loader.dataset
# Get a random sample
random_index = int(np.random.random()*len(dataset))
sample = dataset[random_index]

fig = plt.figure(figsize=(10, 5))
axs = fig.subplots(3, 1, sharex=True, gridspec_kw={"hspace": 0, "height_ratios": [2, 1, 1]})

axs[0].plot((sample["X"][0]/np.amax(sample["X"][0])).T, label='HHZ', linewidth=1)
axs[1].plot(sample["y"].T, label=['P-wave','S-wave','Detection'])

weights = torch.load('.cache/swag-5.pt')['state_dict']
model = SWAG(
        EQTransformer,
        no_cov_mat=True,
        max_num_models=20,
        in_channels= 1
    )
model.load_state_dict(state_dict=weights, strict=False)
model.cuda()

model.sample(0.0)
pred = model(torch.tensor(sample['X'][:1,:], device='cuda').unsqueeze(0)).detach().cpu().numpy()[0]
axs[2].plot(pred.T, label=['P-wave','S-wave','Detection'])

print(pred)

for ax in axs:
    ax.legend()
